## Домашнее задание 6.

1. Попробуйте обучить нейронную сеть с применением одномерных сверток для предсказания сентимента сообщений с твитера на примере https://www.kaggle.com/datasets/arkhoshghalb/twitter-sentiment-analysis-hatred-speech

2. Опишите, какой результат вы получили? Что помогло вам улучшить ее точность?

## Решение

Сначала импортируем все библиотеки, которые будут необходимы при дальнейшем решении задачи.

In [1]:
import torch
import re
import pandas as pd
import numpy as np
import nltk

import torch.nn as nn
import torch.nn.functional as F

from torch.utils.data import DataLoader, Dataset
from string import punctuation
from textblob import TextBlob, Word
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.probability import FreqDist
from itertools import islice
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler

nltk.download("punkt")

[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

Проведем стандартную проверку на наличие видеокарты.

In [2]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda', index=0)

Зададим ряд гиперпараметров, которые будут использоваться в дальнейшем процессе обучения.

In [3]:
max_words = 1500
max_len = 15
num_classes = 1

# Training
epochs = 7
batch_size = 512
embedding_dim = 256
out_channel = 256

Считываем тренировочный датасет, на основе которого будем обучать нейросеть.

In [4]:
DATA_ROOT = '/kaggle/input/twitter-sentiment-analysis-hatred-speech'

df_train = pd.read_csv(DATA_ROOT + "/train.csv")
df_train.head()

,id,label,tweet
0,1,0,@user when a father is dysfunctional and is s...
1,2,0,@user @user thanks for #lyft credit i can't us...
2,3,0,bihday your majesty
3,4,0,#model i love u take with u all the time in ...
4,5,0,factsguide: society now #motivation


В данном случае мы имеем дело с несбалансированным датасетом.

In [5]:
df_train['label'].value_counts()

0    29720
1     2242
Name: label, dtype: int64

Сразу разделим данные на обучающий и валидационный наборы, добавим стратификацию, чтобы сохранить пропорции классов.

In [6]:
X_train, X_val, y_train, y_val = train_test_split(df_train['tweet'], 
                                                  df_train['label'], 
                                                  test_size=0.3, 
                                                  random_state=42, 
                                                  stratify=df_train['label'])

Теперь перейдем к предобработке твитов. Для начала сформируем множество стоп слов.

In [7]:
sw = set(stopwords.words("english"))
# Добавим к стандартному множеству еще одно слово, которое не несет смысловой нагрузки,
# но часто встречается как текстовое представление символа - &amp; 
sw.add('amp')
# Добавим user, так как в данном датасете это является обезличенным 
# упоминанием пользователя в твите
sw.add('user')
sw

{'a',
 'about',
 'above',
 'after',
 'again',
 'against',
 'ain',
 'all',
 'am',
 'amp',
 'an',
 'and',
 'any',
 'are',
 'aren',
 "aren't",
 'as',
 'at',
 'be',
 'because',
 'been',
 'before',
 'being',
 'below',
 'between',
 'both',
 'but',
 'by',
 'can',
 'couldn',
 "couldn't",
 'd',
 'did',
 'didn',
 "didn't",
 'do',
 'does',
 'doesn',
 "doesn't",
 'doing',
 'don',
 "don't",
 'down',
 'during',
 'each',
 'few',
 'for',
 'from',
 'further',
 'had',
 'hadn',
 "hadn't",
 'has',
 'hasn',
 "hasn't",
 'have',
 'haven',
 "haven't",
 'having',
 'he',
 'her',
 'here',
 'hers',
 'herself',
 'him',
 'himself',
 'his',
 'how',
 'i',
 'if',
 'in',
 'into',
 'is',
 'isn',
 "isn't",
 'it',
 "it's",
 'its',
 'itself',
 'just',
 'll',
 'm',
 'ma',
 'me',
 'mightn',
 "mightn't",
 'more',
 'most',
 'mustn',
 "mustn't",
 'my',
 'myself',
 'needn',
 "needn't",
 'no',
 'nor',
 'not',
 'now',
 'o',
 'of',
 'off',
 'on',
 'once',
 'only',
 'or',
 'other',
 'our',
 'ours',
 'ourselves',
 'out',
 'over',
 'o

Сформируем дополнительно список знаков пунктуации.

In [8]:
puncts = set(punctuation)
puncts

{'!',
 '"',
 '#',
 '$',
 '%',
 '&',
 "'",
 '(',
 ')',
 '*',
 '+',
 ',',
 '-',
 '.',
 '/',
 ':',
 ';',
 '<',
 '=',
 '>',
 '?',
 '@',
 '[',
 '\\',
 ']',
 '^',
 '_',
 '`',
 '{',
 '|',
 '}',
 '~'}

Напишем функцию по аналогии с представленной на лекции, которая будет производить предобработку подаваемого текста.

In [9]:
def preprocess_text(txt):
    txt = str(txt)
    # уберем нечитаемые символы типа  ð\x9f¤\x97
    txt = "".join([c for c in txt if ord(c) < 128])
    txt = "".join(c for c in txt if c not in puncts)
    txt = txt.lower()
    # преобразуем отрицания
    txt = re.sub("not\s", "not", txt)
    txt = re.sub("no\s", "no", txt)
    # будем приводить формы к глаголам
    txt = [Word(word).lemmatize('v') for word in txt.split() if word not in sw]
    return " ".join(txt)

Посмотрим на примерах на результат работы предобработки.

In [10]:
X_train.iloc[:10].values

array(['happy bihday to my brother man. needed this mixtape like we need boos. have a good one sach   @user ',
       '  lang to sta the week right :)  #happiness #smile ',
       'note it meditate on it work on it ,but most impoantly trust god for it #icantwaitfohedayhisplansformylifeunfold #grateful  ',
       '@user listening to you this wet mon, ahead of #leedsmillenium gig next month   ð\x9f\x98\x86ð\x9f\x91\x8dð\x9f\x98\x8d #music #ace ',
       '@user @user agreed.. the same is true for  and .. they are overused terms, and as a result, are fast becominâ\x80¦',
       'very exciting! #dubllife #recycle ',
       '#bad times #drink   #nobev ',
       '#ootd #converse #denim #tshi  #shopping  #like4like #l4l #f4f #instagood  ',
       '  #fathersday to the man of my dreams! you sacrificed bachelorhood for a ready-made familyâ\x80¦ ',
       '  #pougalday #pay #saturday #fresh #new #haircut &amp; new #red #car in #style #chillingâ\x80¦ '],
      dtype=object)

In [11]:
X_train.iloc[:10].apply(preprocess_text).values

array(['happy bihday brother man need mixtape like need boo good one sach',
       'lang sta week right happiness smile',
       'note meditate work impoantly trust god icantwaitfohedayhisplansformylifeunfold grateful',
       'listen wet mon ahead leedsmillenium gig next month music ace',
       'agree true overuse term result fast becomin',
       'excite dubllife recycle', 'bad time drink nobev',
       'ootd converse denim tshi shop like4like l4l f4f instagood',
       'fathersday man dream sacrifice bachelorhood readymade family',
       'pougalday pay saturday fresh new haircut new red car style chill'],
      dtype=object)

Теперь преобразуем тексты всех твитов с помощью данной функции.

In [12]:
X_train = X_train.apply(preprocess_text).values
X_val = X_val.apply(preprocess_text).values

Перейдем к реализации процесса токенизации.

In [13]:
train_corpus = " ".join(X_train)
train_corpus = train_corpus.lower()

In [14]:
tokens = word_tokenize(train_corpus)
tokens[:5]

['happy', 'bihday', 'brother', 'man', 'need']

In [15]:
tokens_filtered = [word for word in tokens if word.isalnum()]
dist = FreqDist(tokens_filtered)
tokens_filtered_top = [pair[0] for pair in dist.most_common(max_words-1)]

# Посмотрим на топ 10 слов
tokens_filtered_top[:10]

['love', 'day', 'get', 'happy', 'go', 'time', 'make', 'im', 'u', 'life']

Сформируем словарь, в котором будут храниться наиболее часто встречающиеся слова.

In [16]:
def take(n, iterable):
    return list(islice(iterable, n))

vocabulary = {v: k for k, v in dict(enumerate(tokens_filtered_top, 1)).items()}
take(20, vocabulary.items())

[('love', 1),
 ('day', 2),
 ('get', 3),
 ('happy', 4),
 ('go', 5),
 ('time', 6),
 ('make', 7),
 ('im', 8),
 ('u', 9),
 ('life', 10),
 ('like', 11),
 ('today', 12),
 ('new', 13),
 ('father', 14),
 ('see', 15),
 ('positive', 16),
 ('smile', 17),
 ('thankful', 18),
 ('people', 19),
 ('bihday', 20)]

Запишем функцию преобразования текста в токены.

In [17]:
def text_to_sequence(text, maxlen):
    result = []
    tokens = word_tokenize(text.lower())
    tokens_filtered = [word for word in tokens if word.isalnum()]
    for word in tokens_filtered:
        if word in vocabulary:
            result.append(vocabulary[word])

    padding = [0] * (maxlen-len(result))
    return result[-maxlen:] + padding

In [18]:
%%time
x_train = np.asarray([text_to_sequence(text, max_len) for text in X_train])
x_val = np.asarray([text_to_sequence(text, max_len) for text in X_val])

CPU times: user 5 s, sys: 14.6 ms, total: 5.01 s
Wall time: 5.03 s


In [19]:
x_train[1]

array([165,  69,  76,  77,  17,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0])

Соберем сеть.

In [20]:
class Net(nn.Module):
    def __init__(self, vocab_size=2000, embedding_dim=128, out_channel=128, num_classes=1):
        super().__init__()
        self.dp = nn.Dropout(0.25)
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.conv_1 = nn.Conv1d(embedding_dim, out_channel, kernel_size=2)
        self.conv_2 = nn.Conv1d(embedding_dim, out_channel, kernel_size=3)
        self.pool = nn.MaxPool1d(2)
        self.relu = nn.ReLU()
        self.linear_1 = nn.Linear(out_channel, out_channel // 2)
        self.linear_2 = nn.Linear(out_channel // 2, num_classes)
#         self.linear_1 = nn.Linear(out_channel, num_classes)
        
    def forward(self, x):        
        output = self.embedding(x) # B, L, E
        #                       B  E  L         
        output = output.permute(0, 2, 1)
        output = self.conv_1(output)
        output = self.relu(output)
        output = self.pool(output)

        output = self.conv_2(output)
        output = self.relu(output)
        output = self.pool(output)
        output = torch.max(output, axis=2).values
        output = self.linear_1(output)
        output = self.relu(output)
        output = self.dp(output)
        output = self.linear_2(output)
        output = torch.sigmoid(output)
        return output

Создадим класс датасета и определим даталоадеры.

In [21]:
class DataWrapper(Dataset):
    def __init__(self, data, target, transform=None):
        self.data = torch.from_numpy(data).long()
        self.target = torch.from_numpy(target).long()
        self.transform = transform
        
    def __getitem__(self, index):
        x = self.data[index]
        y = self.target[index]
        
        if self.transform:
            x = self.transform(x)
            
        return x, y
    
    def __len__(self):
        return len(self.data)

In [22]:
train_dataset = DataWrapper(x_train, y_train.values)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

val_dataset = DataWrapper(x_val, y_val.values)
val_loader = DataLoader(val_dataset, batch_size=8, shuffle=True)

Обучаем построенную сеть.

In [23]:
model = Net(vocab_size=max_words, embedding_dim=embedding_dim, out_channel=out_channel)

In [24]:
print(model)
print("Parameters:", sum([param.nelement() for param in model.parameters()]))

Net(
  (dp): Dropout(p=0.25, inplace=False)
  (embedding): Embedding(1500, 256)
  (conv_1): Conv1d(256, 256, kernel_size=(2,), stride=(1,))
  (conv_2): Conv1d(256, 256, kernel_size=(3,), stride=(1,))
  (pool): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (relu): ReLU()
  (linear_1): Linear(in_features=256, out_features=128, bias=True)
  (linear_2): Linear(in_features=128, out_features=1, bias=True)
)
Parameters: 745217


In [25]:
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
criterion = nn.BCELoss()

In [26]:
model = model.to(device)
model.train()
th = 0.5

train_loss_history = []
test_loss_history = []


for epoch in range(epochs):  
    running_items, running_right = 0.0, 0.0
    for i, data in enumerate(train_loader, 0):
        inputs, labels = data[0].to(device), data[1].to(device)
        
        # обнуляем градиент
        optimizer.zero_grad()
        outputs = model(inputs)
        
        loss = criterion(outputs, labels.float().view(-1, 1))
        loss.backward()
        optimizer.step()

        # подсчет ошибки на обучении
        loss = loss.item()
        running_items += len(labels)
        # подсчет метрики на обучении
        pred_labels = torch.squeeze((outputs > th).int())
        running_right += (labels == pred_labels).sum()
        
    # выводим статистику о процессе обучения
    model.eval()
    
    print(f'Epoch [{epoch + 1}/{epochs}]. ' \
            f'Step [{i + 1}/{len(train_loader)}]. ' \
            f'Loss: {loss:.3f}. ' \
            f'Acc: {running_right / running_items:.3f}', end='. ')
    running_loss, running_items, running_right = 0.0, 0.0, 0.0
    train_loss_history.append(loss)

        # выводим статистику на тестовых данных
    test_running_right, test_running_total, test_loss = 0.0, 0.0, 0.0
    for j, data in enumerate(val_loader):
        test_labels = data[1].to(device)
        test_outputs = model(data[0].to(device))
        
        # подсчет ошибки на тесте
        test_loss = criterion(test_outputs, test_labels.float().view(-1, 1))
        # подсчет метрики на тесте
        test_running_total += len(data[1])
        pred_test_labels = torch.squeeze((test_outputs > th).int())
        test_running_right += (test_labels == pred_test_labels).sum()
    
    test_loss_history.append(test_loss.item())
    print(f'Test loss: {test_loss:.3f}. Test acc: {test_running_right / test_running_total:.3f}')
    
    model.train()
        
print('Training is finished!')

Epoch [1/7]. Step [44/44]. Loss: 0.250. Acc: 0.921. Test loss: 0.346. Test acc: 0.930
Epoch [2/7]. Step [44/44]. Loss: 0.163. Acc: 0.930. Test loss: 0.029. Test acc: 0.930
Epoch [3/7]. Step [44/44]. Loss: 0.184. Acc: 0.940. Test loss: 0.004. Test acc: 0.944
Epoch [4/7]. Step [44/44]. Loss: 0.116. Acc: 0.960. Test loss: 0.112. Test acc: 0.950
Epoch [5/7]. Step [44/44]. Loss: 0.056. Acc: 0.977. Test loss: 0.606. Test acc: 0.950
Epoch [6/7]. Step [44/44]. Loss: 0.056. Acc: 0.987. Test loss: 0.072. Test acc: 0.948
Epoch [7/7]. Step [44/44]. Loss: 0.029. Acc: 0.989. Test loss: 0.000. Test acc: 0.951
Training is finished!


In [27]:
oversample = RandomOverSampler(sampling_strategy='minority')
newdata, newtargets = oversample.fit_resample(x_train, y_train)

In [28]:
trainnew_dataset = DataWrapper(newdata, newtargets.values)
trainnew_loader = DataLoader(trainnew_dataset, batch_size=batch_size, shuffle=True)

In [29]:
model = Net(vocab_size=max_words, embedding_dim=embedding_dim, out_channel=out_channel)

In [30]:
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
criterion = nn.BCELoss()

In [31]:
model = model.to(device)
model.train()
th = 0.5

train_loss_history = []
test_loss_history = []


for epoch in range(epochs):  
    running_items, running_right = 0.0, 0.0
    for i, data in enumerate(trainnew_loader, 0):
        inputs, labels = data[0].to(device), data[1].to(device)
        
        # обнуляем градиент
        optimizer.zero_grad()
        outputs = model(inputs)
        
        loss = criterion(outputs, labels.float().view(-1, 1))
        loss.backward()
        optimizer.step()

        # подсчет ошибки на обучении
        loss = loss.item()
        running_items += len(labels)
        # подсчет метрики на обучении
        pred_labels = torch.squeeze((outputs > th).int())
        running_right += (labels == pred_labels).sum()
        
    # выводим статистику о процессе обучения
    model.eval()
    
    print(f'Epoch [{epoch + 1}/{epochs}]. ' \
            f'Step [{i + 1}/{len(trainnew_loader)}]. ' \
            f'Loss: {loss:.3f}. ' \
            f'Acc: {running_right / running_items:.3f}', end='. ')
    running_loss, running_items, running_right = 0.0, 0.0, 0.0
    train_loss_history.append(loss)

        # выводим статистику на тестовых данных
    test_running_right, test_running_total, test_loss = 0.0, 0.0, 0.0
    for j, data in enumerate(val_loader):
        test_labels = data[1].to(device)
        test_outputs = model(data[0].to(device))
        
        # подсчет ошибки на тесте
        test_loss = criterion(test_outputs, test_labels.float().view(-1, 1))
        # подсчет метрики на тесте
        test_running_total += len(data[1])
        pred_test_labels = torch.squeeze((test_outputs > th).int())
        test_running_right += (test_labels == pred_test_labels).sum()
    
    test_loss_history.append(test_loss.item())
    print(f'Test loss: {test_loss:.3f}. Test acc: {test_running_right / test_running_total:.3f}')
    
    model.train()
        
print('Training is finished!')

Epoch [1/7]. Step [82/82]. Loss: 0.118. Acc: 0.846. Test loss: 0.002. Test acc: 0.911
Epoch [2/7]. Step [82/82]. Loss: 0.039. Acc: 0.976. Test loss: 0.001. Test acc: 0.929
Epoch [3/7]. Step [82/82]. Loss: 0.060. Acc: 0.984. Test loss: 0.000. Test acc: 0.948
Epoch [4/7]. Step [82/82]. Loss: 0.021. Acc: 0.987. Test loss: 0.145. Test acc: 0.936
Epoch [5/7]. Step [82/82]. Loss: 0.012. Acc: 0.988. Test loss: 1.639. Test acc: 0.934
Epoch [6/7]. Step [82/82]. Loss: 0.034. Acc: 0.988. Test loss: 0.000. Test acc: 0.933
Epoch [7/7]. Step [82/82]. Loss: 0.014. Acc: 0.986. Test loss: 0.000. Test acc: 0.926
Training is finished!


Оверсэмплинг не позволил добиться улучшения качества модели.

В процессе экспериментов попробовал:
*  изменять параметры лемматизации и в целом процесс предобработки;
*  изменять гиперпараметры модели;
*  изменять структуру сети;
*  изменять оптимизаторы и их гиперпараметры.

Тем не менее существенных улучшений мне добиться не удалось, тестовая метрика accuracy в большинстве случаев остается в диапазоне 0.945-0.955, а главной проблемой, на мой взгляд, является переобучение.